<a href="https://colab.research.google.com/github/aisaza609/Datos_a_la_U_grupo_2/blob/main/Normalizacion_Dataset_Pasto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import io
from io import BytesIO
import random
import ipywidgets as widgets
from IPython.display import display

%matplotlib inline

# DATASET PASTO NIVEL 1

El dataset del hospital de pasto, corresponde al hospital: Empresa Social del Estado Pasto Salud que es de nivel, que corresponde a la mayoria de hospital de las regiones rurales del pais.

In [30]:
# URL directa al archivo en formato .xlsx en GitHub
url_excel = 'https://raw.githubusercontent.com/aisaza609/Datos_a_la_U_grupo_2/main/Dataset%20de%20datos%20abiertos/Hospital_Pasto_v1.xlsx'

# Descargar y cargar el archivo .xlsx
response_excel = requests.get(url_excel)
if response_excel.status_code == 200:
    df_pasto = pd.read_excel(BytesIO(response_excel.content), engine='openpyxl')
    # Mostrar las primeras filas para verificar

else:
    print("No se pudo descargar el archivo .xlsx.")


In [31]:
df_pasto.head()

,IPS,FechaAsignacionRegistro,Sexo,Edad,Hora_Salida,Objeto_Remision,Impresiones_Diagnosticas,NivelHospital,NiveldeAtencionNecesario,Traslado,RangoEdad
0,CS LORENZO,01/07/2020 11:20:00 AM,M,70,11:15,OTRO,Diagnóstico Pincipal: K069 - TRASTORNO NO ESPE...,1,NaN,NaN,3
1,CS OBONUCO,01/07/2020 11:22:00 AM,M,24,11:16,OTRO,Diagnóstico Pincipal: K047 - ABSCESO PERIAPICA...,1,NaN,NaN,2
2,CS ENCANO,01/07/2020 11:25:00 AM,F,37,11:24,MEDICINA INTERNA,Diagnóstico Pincipal: I443 - OTROS TIPOS DE BL...,1,2.0,Traslado a Nivel 2,2
3,CS TAMASAGRA,01/07/2020 11:28:00 AM,F,3,11:26,PEDIATRIA,Diagnóstico Pincipal: E45X - RETARDO DEL DESAR...,1,2.0,Traslado a Nivel 2,1
4,CS TAMASAGRA,01/07/2020 11:31:00 AM,M,55,11:28,ORTOPEDIA,Diagnóstico Pincipal: M751 - SINDROME DE MANGUITO,1,2.0,Traslado a Nivel 2,3


## Normalizacion de Los DATOS

In [32]:
df_pasto.Impresiones_Diagnosticas.value_counts() #mostramos la informacion de las columnas

,count
Impresiones_Diagnosticas,
Diagnóstico Pincipal: I10X - HIPERTENSION ESENCIAL,3146
Diagnóstico Pincipal: H527 - TRASTORNO DE LA REFRA,2266
Diagnóstico Pincipal: Z359 - SUPERVISION DE EMBARA,1278
Diagnóstico Pincipal: H521 - MIOPIA ;,960
Diagnóstico Pincipal: K081 - PERDIDA DE DIENTES DE,724
...,...
Diagnóstico Pincipal: M183 - OTRAS ARTROSIS POSTRA,2
Diagnóstico Pincipal: S024 - FRACTURA DEL MALAR Y,2
Diagnóstico Pincipal: F149 - TRASTORNOS MENTALES Y,2


In [33]:
# Usamos str.extract() con una expresión regular para extraer el código y el nombre del diagnóstico
df_pasto[['CodigoDiagnostico', 'Nombre Diagnostico']] = df_pasto['Impresiones_Diagnosticas'].str.extract(r'Diagnóstico Pincipal: (\w+) - (.+)')

# Mostramos las primeras filas para verificar
print(df_pasto[['Impresiones_Diagnosticas', 'CodigoDiagnostico', 'Nombre Diagnostico']].head())

                            Impresiones_Diagnosticas CodigoDiagnostico  \
0  Diagnóstico Pincipal: K069 - TRASTORNO NO ESPE...              K069   
1  Diagnóstico Pincipal: K047 - ABSCESO PERIAPICA...              K047   
2  Diagnóstico Pincipal: I443 - OTROS TIPOS DE BL...              I443   
3  Diagnóstico Pincipal: E45X - RETARDO DEL DESAR...              E45X   
4  Diagnóstico Pincipal: M751 - SINDROME DE MANGUITO              M751   

      Nombre Diagnostico  
0  TRASTORNO NO ESPECIFI  
1  ABSCESO PERIAPICAL SI  
2  OTROS TIPOS DE BLOQUE  
3  RETARDO DEL DESARROLL  
4   SINDROME DE MANGUITO  


PARA NORMALIZAR EL DATASET HACEMOS EL MISMO PROCESO ELIMINAMOS LOS DATOS QUE NO SON NECESARIOS POR AHORA PARA LA INTEROPERABILIDAD

In [34]:
# Cambiar el nombre de la columna 'Objeto_Remision' a 'Servicio' en el DataFrame tg_pitalito
df_pasto.rename(columns={'Objeto_Remision': 'Servicio'}, inplace=True)


CREAMOS LOS ATRIBUTOS TIPO DE DOC, CEDULA, CODIGO DE INGRESO

### tipo de identificación

In [35]:
# Define las condiciones para cada tipo de identificación en función de la edad
conditions = [
    (df_pasto['Edad'] < 18),  # Menores de 18 años
    (df_pasto['Edad'] >= 18)  # Mayores de 18 años
]

# Define los valores que correspondan a cada condición
choices = ['TI', 'CC']

# Crea la nueva columna usando np.select
df_pasto['tipo de identificación'] = np.select(conditions, choices, default='Desconocido')

# Muestra las primeras filas para verificar
print(df_pasto[['Edad', 'tipo de identificación']].head())


   Edad tipo de identificación
0    70                     CC
1    24                     CC
2    37                     CC
3     3                     TI
4    55                     CC


### NDocumento

Creamos numeros de documentos para simular la identidad de pacientes

In [36]:
# Generar una lista de números únicos de 10 dígitos con la misma cantidad de filas que el DataFrame
n_documentos = random.sample(range(1000000000, 9999999999), len(df_pasto))

# Asignar la lista generada a la nueva columna 'NDocumento'
df_pasto['NDocumento'] = n_documentos

# Muestra las primeras filas para verificar
print(df_pasto[['NDocumento']].head())

   NDocumento
0  7042205939
1  9380617426
2  4381334137
3  8279654440
4  5568749166


### cdIngreso

Hacemos numeros aleatoros de 7 digitos para asignar un codigo a cada Historia clinica

In [37]:
# Generar una lista de números únicos de 7 dígitos con la misma cantidad de filas que el DataFrame para el codigo de ingreso
n_ingreso = random.sample(range(1000000000, 9999999999), len(df_pasto))

# Asignar la lista generada a la nueva columna 'cdIngreso'
df_pasto['Ingreso'] = n_documentos

# Muestra las primeras filas para verificar
print(df_pasto[['Ingreso']].head())

      Ingreso
0  7042205939
1  9380617426
2  4381334137
3  8279654440
4  5568749166


### Eliminacion de Columnas

In [38]:
df_pasto.drop(['Hora_Salida', 'Impresiones_Diagnosticas', 'RangoEdad'], axis=1, inplace=True)

In [39]:
df_pasto.rename(columns={'IPS': 'Entidad'}, inplace=True)
df_pasto.rename(columns={'Nombre Diagnostico': 'NombreDiagnostico'}, inplace=True)
df_pasto.rename(columns={'FechaAsignacionRegistro': 'FechaIngreso'}, inplace=True)


### Dataset CE-10

In [40]:
# URL directa al archivo en GitHub
url3 = 'https://raw.githubusercontent.com/aisaza609/Datos_a_la_U_grupo_2/main/Datasets%20externos/TablaReferencia_CIE10__1.xlsx'

# Descargar el archivo desde GitHub
response = requests.get(url3)
if response.status_code == 200:
    # Leer el contenido con pandas, especificando el motor como 'openpyxl'
    excel_data = pd.read_excel(BytesIO(response.content), engine='openpyxl')
    # Mostrar las primeras filas para verificar

else:
    print("No se pudo descargar el archivo.")
excel_data.head()

,Tabla,Codigo,Nombre,Descripcion,Habilitado,Aplicacion,IsStandardGEL,IsStandardMSPS,Extra_I:AplicaASexo,Extra_II:EdadMinima,...,Extra_V,Extra_VI:Capitulo,Extra_VII:Grupo,Extra_VIII:SubGrupo,Extra_IX:Categoria,Extra_X:Sexo,ValorRegistro,UsuarioResponsable,Fecha_Actualizacion,IsPublicPrivate
0,CIE10,A000,"COLERA DEBIDO A VIBRIO CHOLERAE 01, BIOTIPO CH...",COLERA,SI,NaN,False,False,3,0,...,CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITARIA...,A00,NaN,1.0,0.0,A,NaN,NaN,2023-02-04 09:43:26 PM,NaN
1,CIE10,A001,"COLERA DEBIDO A VIBRIO CHOLERAE 01, BIOTIPO EL...",COLERA,SI,NaN,False,False,3,0,...,CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITARIA...,A00,NaN,1.0,NaN,A,NaN,NaN,2023-02-04 09:43:26 PM,NaN
2,CIE10,A009,"COLERA, NO ESPECIFICADO",COLERA,SI,NaN,False,False,3,0,...,CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITARIA...,A00,NaN,1.0,NaN,A,NaN,NaN,2023-02-04 09:43:26 PM,NaN
3,CIE10,A010,FIEBRE TIFOIDEA,FIEBRES TIFOIDEA Y PARATIFOIDEA,SI,NaN,False,False,3,0,...,CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITARIA...,A01,NaN,1.0,NaN,A,NaN,NaN,2023-02-04 09:43:26 PM,NaN
4,CIE10,A011,FIEBRE PARATIFOIDEA A,FIEBRES TIFOIDEA Y PARATIFOIDEA,SI,NaN,False,False,3,0,...,CIERTAS ENFERMEDADES INFECCIOSAS Y PARASITARIA...,A01,NaN,1.0,NaN,A,NaN,NaN,2023-02-04 09:43:26 PM,NaN


In [41]:
excel_data = excel_data.iloc[:, 1:4]

# Mostrar el resultado para verificar
excel_data.head()

,Codigo,Nombre,Descripcion
0,A000,"COLERA DEBIDO A VIBRIO CHOLERAE 01, BIOTIPO CH...",COLERA
1,A001,"COLERA DEBIDO A VIBRIO CHOLERAE 01, BIOTIPO EL...",COLERA
2,A009,"COLERA, NO ESPECIFICADO",COLERA
3,A010,FIEBRE TIFOIDEA,FIEBRES TIFOIDEA Y PARATIFOIDEA
4,A011,FIEBRE PARATIFOIDEA A,FIEBRES TIFOIDEA Y PARATIFOIDEA


Considerando que este dataset no tiene descripcion del diagnostico usamos el datataset de CE-10 para segun el codgio llenar el nombre

In [42]:
# Realizar un merge entre df_pasto y excel_data en base a 'CodigoDiagnostico' de df_pasto y 'Codigo' de excel_data,
# renombrando la columna 'Descripcion' como 'DxSindromatico' directamente en el merge
df_pasto = df_pasto.merge(excel_data[['Codigo', 'Descripcion']].rename(columns={'Descripcion': 'DxSindromatico'}),
                          left_on='CodigoDiagnostico', right_on='Codigo', how='left')

# Eliminar la columna 'Codigo' adicional generada tras el merge
df_pasto = df_pasto.drop(columns=['Codigo'])

# Verificar el resultado
print(df_pasto[['CodigoDiagnostico', 'DxSindromatico']].head())




  CodigoDiagnostico                                     DxSindromatico
0              K069  OTROS TRASTORNOS DE LA ENCIA Y DE LA ZONA EDEN...
1              K047  ENFERMEDADES DE LA PULPA Y DE LOS TEJIDOS PERI...
2              I443  BLOQUEO AURICULOVENTRICULAR Y DE RAMA IZQUIERD...
3              E45X  RETARDO DEL DESARROLLO DEBIDO A DESNUTRICION P...
4              M751                                LESIONES DEL HOMBRO


In [47]:
orden_columnas = [
    'tipo de identificación',
    'NDocumento',
    'Sexo',
    'Edad',
    'Ingreso',
    'FechaIngreso',
    'Entidad',
    'CodigoDiagnostico',
    'NombreDiagnostico',
    'DxSindromatico',
    'NiveldeAtencionNecesario',
    'Servicio',
    'NivelHospital',
    'Traslado'
]
# Reordenar el DataFrame df_pasto con el nuevo orden de columnas
df_pasto = df_pasto[orden_columnas]


In [48]:
df_pasto


,tipo de identificación,NDocumento,Sexo,Edad,Ingreso,FechaIngreso,Entidad,CodigoDiagnostico,NombreDiagnostico,DxSindromatico,NiveldeAtencionNecesario,Servicio,NivelHospital,Traslado
0,CC,7042205939,M,70,7042205939,01/07/2020 11:20:00 AM,CS LORENZO,K069,TRASTORNO NO ESPECIFI,OTROS TRASTORNOS DE LA ENCIA Y DE LA ZONA EDEN...,NaN,OTRO,1,NaN
1,CC,9380617426,M,24,9380617426,01/07/2020 11:22:00 AM,CS OBONUCO,K047,ABSCESO PERIAPICAL SI,ENFERMEDADES DE LA PULPA Y DE LOS TEJIDOS PERI...,NaN,OTRO,1,NaN
2,CC,4381334137,F,37,4381334137,01/07/2020 11:25:00 AM,CS ENCANO,I443,OTROS TIPOS DE BLOQUE,BLOQUEO AURICULOVENTRICULAR Y DE RAMA IZQUIERD...,2.0,MEDICINA INTERNA,1,Traslado a Nivel 2
3,TI,8279654440,F,3,8279654440,01/07/2020 11:28:00 AM,CS TAMASAGRA,E45X,RETARDO DEL DESARROLL,RETARDO DEL DESARROLLO DEBIDO A DESNUTRICION P...,2.0,PEDIATRIA,1,Traslado a Nivel 2
4,CC,5568749166,M,55,5568749166,01/07/2020 11:31:00 AM,CS TAMASAGRA,M751,SINDROME DE MANGUITO,LESIONES DEL HOMBRO,2.0,ORTOPEDIA,1,Traslado a Nivel 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30167,CC,5326644049,F,53,5326644049,01/20/2020 10:41:00 AM,CS OBONUCO,E119,DIABETES MELLITUS NO,DIABETES MELLITUS NO INSULINODEPENDIENTE,2.0,MEDICINA INTERNA,1,Traslado a Nivel 2
30168,CC,7312237021,F,53,7312237021,01/20/2020 10:43:00 AM,CS OBONUCO,E119,DIABETES MELLITUS NO,DIABETES MELLITUS NO INSULINODEPENDIENTE,3.0,OFTALMOLOGIA,1,Traslado a Nivel 3
30169,CC,2681137859,M,22,2681137859,01/20/2020 10:43:00 AM,CS SAN VICENTE,L910,CICATRIZ QUELOIDE ;,TRASTORNOS HIPERTROFICOS DE LA PIEL,3.0,DERMATOLOGIA,1,Traslado a Nivel 3
30170,CC,8380143449,F,91,8380143449,01/20/2020 10:44:00 AM,CS PROGRESO,N189,INSUFICIENCIA RENAL C,ENFERMEDAD RENAL CRONICA,3.0,NEFROLOGIA,1,Traslado a Nivel 3


In [49]:
# Guardar un DataFrame en un archivo CSV
df_pasto.to_csv('Dataset_pasto_normalizado.csv', index=False)